# util

> Collection of helper functions to facilitate plotting

In [ ]:
#| default_exp util

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

Functions in this module depend on `numpy`/`pandas`, `anndata`, and `scanpy`.

In [ ]:
#| export
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

import scanpy as sc
import anndata as ad

from matplotlib.patches import Patch

from samap.mapping import SAMAP

In [ ]:
#| hide
from nbdev.showdoc import *

from fastcore.test import *
from fastcore.docments import docments

In [ ]:
#| export
def procrustes(x:str, # input string
               appropriate_length:int=50, # desired length
               pad_with:str=" ", # character to pad with
               side:str="right" # which side to pad on ("left", "right")
              )->str: # string with desired length
    "A function to regulate string length."
    if len(x) > appropriate_length:
        return x[:appropriate_length]
    if len(x) < appropriate_length:
        to_pad = appropriate_length - len(x)
        pad = "".join([pad_with] * to_pad)
        if side == "right":
            x = x + pad
        elif side == "left":
            x = pad + x
        else:
            print("Invalid side argument; returning string as-is.")
    return x

We are primarily going to be working with non-model species, so the gene names will always be of the form

`XLOC_123456 | emapper-name-or-description-if-we're-lucky`

or something similar. This means that we could have extreme variation in the actual length of a gene "name"; this will make it very hard to put gene names on axes as it will distort figure sizes. I wrote a function to either trim or pad strings; even though axis labels are not in monospace fonts, it is much easier to visually reconcile strings with lengths in the same order of magnitude.

In [ ]:
too_short = 'Niko'
just_right = 'Theseus'
too_tall = 'The Mountain'

In [ ]:
assert procrustes(just_right, appropriate_length=7) == 'Theseus'
assert procrustes(too_short, appropriate_length=7) == 'Niko   '
assert procrustes(too_tall, appropriate_length=7) == 'The Mou'

<a id='grouped_obs_mean'></a>

In [ ]:
#| export
def grouped_obs_mean(adata:ad.AnnData, # AnnData object to analyse
                        group_key:str, # `.obs` category to group by
                        layer:str=None # layer to use. If none, use `.X`
                    )->pd.DataFrame: # a groups$\times$genes dataframe with the average expression
    "Helper function to calculate average expression per group in an `AnnData` object."
    if layer is not None:
        getX = lambda x: x.layers[layer]
    else:
        getX = lambda x: x.X

    grouped = adata.obs.groupby(group_key)
    out = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float64),
        columns=list(grouped.groups.keys()),
        index=adata.var_names,
    )

    for group, idx in grouped.indices.items():
        X = getX(adata[idx])
        out[group] = np.ravel(X.mean(axis=0, dtype=np.float64))
    return out

Many tasks in single-cell analysis require us to know the average expression of a gene in a certain
group of cells. While `scanpy` _does_ perform that task behind the scenes for, e.g. dotplots, this
is not functionality that is exposed to the users. This is an implementation based on [ivirshup's
answer](https://github.com/theislab/scanpy/issues/181#issuecomment-534867254) to a scanpy issue.

In [ ]:
%%bash
#| hide
FILE="../example_data/hydra.h5ad"
if test -f "$FILE"
then
    echo "$FILE exists."
else
    cd ..
    wget https://zenodo.org/record/8129708/files/example_data.tar.gz
    tar -xzf example_data.tar.gz
    cd ../scripts
fi

In [ ]:
adata = sc.read_h5ad('../example_data/hydra.h5ad')

cluster_means = grouped_obs_mean(adata, group_key='Cluster')

If $G$ is the number of genes and $C$ the number of unique clusters in the `group_key`, the returned array should have the shape $G \times C$:

In [ ]:
no_genes = adata.shape[1]
no_clusters = len(np.unique(adata.obs['Cluster']))
assert cluster_means.shape == (no_genes, no_clusters)

Additionally, each column of the array should contain the average detected expression for cells in that cluster:

In [ ]:
belong_to_ecEp_SC2 = adata.obs['Cluster'] == 'ecEp_SC2'
ecEp_SC2_average = np.mean(adata[belong_to_ecEp_SC2].X, axis=0)
ecEp_SC2_average = np.array(ecEp_SC2_average)[0]

assert all(np.isclose(cluster_means['ecEp_SC2'], ecEp_SC2_average))

In [ ]:
#| export
def grouped_obs_present(adata, group_key, layer=None):
    """
    Helper function to calculate how many cells express each gene per group in an `AnnData` object.

    Parameters
    ----------
    adata : AnnData
        AnnData object to analyse.
    group_key : str
        `.obs` category to group by.
    layer : str, optional
        Layer to use. If none, use `.X`.
    
    Returns
    -------
    pd.DataFrame
        A clusters$\times$genes dataframe with the number of expressing cells per cluster.
    """
    if layer is not None:
        getX = lambda x: x.layers[layer]
    else:
        getX = lambda x: x.X

    grouped = adata.obs.groupby(group_key)
    out = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float64),
        columns=list(grouped.groups.keys()),
        index=adata.var_names,
    )

    for group, idx in grouped.indices.items():
        X = getX(adata[idx])
        out[group] = np.ravel((X > 0).sum(axis=0, dtype=np.float64))
    return out

Another critical value to know when making dotplots is the fraction of cells expressing a gene in a certain cluster. Again, `scanpy` performs that task without exposing it to the users. Similar to [`grouped_obs_mean`](#grouped_obs_mean) this is an implementation based on [ivirshup's answer](https://github.com/theislab/scanpy/issues/181#issuecomment-534867254) to a scanpy issue. Here we calculate the sum of cells expressing a gene, a table we can use to calculate the fraction later.

In [ ]:
num_expressing = grouped_obs_present(adata, group_key='Cluster')

If $G$ is the number of genes and $C$ the number of unique clusters in the `group_key`, the returned array should have the shape $G \times C$:

In [ ]:
assert num_expressing.shape == (no_genes, no_clusters)

Additionally, each column of the array should contain the percentage of cells expressing each gene in that cluster:

In [ ]:
belong_to_ecEp_SC2 = adata.obs['Cluster'] == 'ecEp_SC2'
ecEp_SC2_expr = np.sum(adata[belong_to_ecEp_SC2].X>0, axis=0)
ecEp_SC2_expr = np.array(ecEp_SC2_expr)[0]

assert all(num_expressing['ecEp_SC2'] == ecEp_SC2_expr)

In [ ]:
#| export
def grouped_obs_percent(adata, group_key, layer=None):
    """
    Helper function to calculate what percentage of cells express each gene per group in an
    `AnnData` object.

    Parameters
    ----------
    adata : AnnData
        AnnData object to analyse.
    group_key : str
        `.obs` category to group by.
    layer : str, optional
        Layer to use. If none, use `.X`.
    
    Returns
    -------
    pd.DataFrame
        A clusters$\times$genes dataframe with the percentage of expressing cells per cluster.
    """
    num_expressing = grouped_obs_present(adata, group_key, layer=layer)
    no_cells_per_cluster = adata.obs[group_key].value_counts()
    return num_expressing / no_cells_per_cluster

Calculating the fraction of cells is of course very straightforward once we have counted the number of cells that express the gene as well as the total number of cells in a cluster.

In [ ]:
frac_expressing = grouped_obs_percent(adata, group_key='Cluster')
# use the counts and number of cells we calculated before
frac_ecEp_SC2 = ecEp_SC2_expr / np.sum(belong_to_ecEp_SC2)

assert all(np.isclose(frac_expressing['ecEp_SC2'], frac_ecEp_SC2))

### Highlighting clusters

Dimensionality reduction plots can often be rather busy, and searching for the correct cluster can
be a bit of a hassle. It would be great if we could highlight the cluster of interest without losing
the rest of the clustering information; for instance by drawing a circle around the cluster to
highlight.

In [ ]:
#| export
def find_center(coords):
    """
    A function that estimates a Gaussian probability density for the input data and returns the
    mode. From https://stackoverflow.com/a/60185876.

    Parameters
    ----------
    coords : np.ndarray
        A 2D array with X, Y-coordinates from xs, ys.

    Returns
    -------
    float
        The X-coordinate of the mode.
    float
        The Y-coordinate of the mode.
    """
    kernel = gaussian_kde(coords.T)
    min_x, min_y = np.min(coords, axis=0)
    max_x, max_y = np.max(coords, axis=0)
    grid_xs, grid_ys = np.mgrid[min_x:max_x:50j, min_y:max_y:50j]
    positions = np.vstack(
        [grid_xs.ravel(), grid_ys.ravel()]
    )  # 2-dim array with X, Y-coordinates from xs, ys
    Z = np.reshape(kernel(positions).T, grid_xs.shape)  # get densities

    idx = np.unravel_index(np.argmax(Z), Z.shape)
    return grid_xs[idx], grid_ys[idx]

A heuristic to achieve this is to pretend the cluster points are a Gaussian cloud on
UMAP/tSNE/PCA/\<your favorite embedding\> space, and take the position with the highest density (the
mode of the 2D distribution). This function is inspired from a [StackOverflow
answer](https://stackoverflow.com/a/60185876), and mostly a wrapper around the [Gaussian
KDE](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html) function
from `scikit-learn`.

In [ ]:
# create a 2D Gaussian dataset
x = np.random.normal(loc=2, scale=1, size=1000)
y = np.random.normal(loc=-2, scale=1, size=1000)

coords = np.array([x, y]).T

In [ ]:
coords_center = find_center(coords)

We'd expect the mode of the kernel density estimate to be very close to the true mean of the data.
Since this is for plotting purposes we don't need to be extremely specific. See
`plot:highlighted_dimplot` for a demonstration of the function.

In [ ]:
assert np.isclose(coords_center[0], 2, rtol=0.2)
assert np.isclose(coords_center[1], -2, rtol=0.2)

In [ ]:
def map_fine_to_coarse(sm, species, fine, coarse, plot=sc.pl.umap, include_coarse=False):
    """
    Extract the mapping of fine to coarse clusters from a SAMap object.

    Parameters
    ----------
    sm : sm.maps.SAMAP
        SAMAP object to process.
    species : str
        Species ID of the correct SAM object.
    fine : str
        Fine clustering slot name.
    coarse : str
        Coarse clustering slot name.
    plot : function, optional
        Plotting function to use; this will correctly set the colors (default: `sc.pl.umap`).
    include_coarse : bool, optional
        If True, preface the fine cluster names with the coarse cluster names (default: `False`).

    Returns
    -------
    fine_to_coarse: pd.DataFrame
        A dataframe with the mapping of fine to coarse clusters.
    lut: dict
        A dictionary with the colors for each coarse cluster.
    handles: list
        A list of `matplotlib.patches.Patch` objects with the colors for each coarse cluster.
    """
    fine_to_coarse = (
        sm.sams[species]
        .adata.obs[[fine, coarse]]
        .drop_duplicates()
        .reset_index(drop=True)
    )

    plot(sm.sams[species].adata, color=coarse)

    lut = dict(
        zip(
            sm.sams[species].adata.obs[coarse].cat.categories,
            sm.sams[species].adata.uns[coarse + "_colors"],
        )
    )
    handles = [Patch(facecolor=lut[name]) for name in lut]
    if include_coarse:
        fine_to_coarse[fine] = (
            species
            + "_"
            + fine_to_coarse[coarse].astype(str)
            + "_"
            + fine_to_coarse[fine].astype(str)
        )
    else:
        fine_to_coarse[fine] = species + "_" + fine_to_coarse[fine].astype(str)
    return fine_to_coarse, lut, handles

We often 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()